In [1]:
# LOAD_NAME = 'rad5cds_local.max.ds2to11_adaptive_wd0.1.pth'
LOAD_NAME = 'rad5cds_local.max.ds2to11_adaptive_wd0.01_25n.pth'
# LOAD_NAME = 'rad5cds_local.max.ds2to11_adaptive_wd0.1_accum2_steps4500_51n.pth'


# LOAD_NAME = 'rad5cds_local.max_adaptive.pth'


# reads_dataset_name = "hsa.dRNASeq.HeLa.polyA.REL5.long.1"
# reads_dataset_name = "hsa.dRNASeq.HeLa.polyA.REL5.long.2"
reads_dataset_name = "hsa_dRNA_HeLa_NoArs_polyA_5P_1"
# reads_dataset_name = "ds1ds2"

# reads_dataset_preprocessing = 'cds_local.max2/adaptive'
# reads_dataset_preprocessing = 'cds_local.max/adaptive'
reads_dataset_preprocessing = 'adaptive/adaptive'
# reads_dataset_preprocessing = 'original'

dataset_save_name = reads_dataset_name

SEQ_LENGTH = 24
if SEQ_LENGTH != 200:
    dataset_save_name = dataset_save_name + "_" + str(SEQ_LENGTH) + 'n' + "_" 

log_number = 9

LOG_OUTPUT_LOCATION = "results_logs/{}_trained_{}_tested_{}.txt".format(LOAD_NAME, reads_dataset_preprocessing.replace('/', '.'), log_number)

dataset_save_name = dataset_save_name + reads_dataset_preprocessing + "_test"
dataset_save_name

'hsa_dRNA_HeLa_NoArs_polyA_5P_1_24n_adaptive/adaptive_test'

In [2]:
import comet_ml
from torch import cuda
cuda.is_available()

/home/jovyan/my-conda-envs/decay3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
import sys
import argparse
import datetime
import numpy as np
from Bio import SeqIO
from math import ceil

from sklearn.model_selection import train_test_split

from raw_to_input import *
from raw_to_input import reformat, parse_data, parse_neg


def load_data(upstream5p, downstream3p, within, rbpfile, negfile, rep, onehot):
    """Parses and processes data from input FASTA and BED files and returns it in train and testing portions.
    @param upstream5p
    """
    # I do not use the RBPs now, but leaving the code here for later
    rbp_dict = get_rbp_dict(rbpfile)

    x, rbps, y = [], [], []
    # Iterate through files and append sequences and corresponding labels and region information to above data arrays
    parse_data(upstream5p, x, y, rbps, rbp_dict, 5, rep, onehot, length=SEQ_LENGTH)
    print(len(y), ' processed upstream5p; ', 'currect label balance: ', y.count(1), ' ones, ', y.count(0), ' zeros')
    parse_data(downstream3p, x, y, rbps, rbp_dict, 3, rep, onehot, length=SEQ_LENGTH)
    print(len(y), ' processed downstream3p; ', 'currect label balance: ', y.count(1), ' ones, ', y.count(0), ' zeros')
    parse_data(within, x, y, rbps, rbp_dict, "w", rep, onehot, length=SEQ_LENGTH)
    print(len(y), ' processed within; ', 'currect label balance: ', y.count(1), ' ones, ', y.count(0), ' zeros')

    x_neg, rbps_neg, y_neg = [], [], []
    
    # generate negatives
    # in each cycle there we add 1 random positive
    while y.count(1) > y_neg.count(0):
        parse_neg(negfile, x_neg, y_neg, rbps_neg, rbp_dict, rep, onehot, length=SEQ_LENGTH)
        print(len(y_neg), ' processed negfile; ', 'current label balance: ', y.count(1), ' ones, ', y_neg.count(0), ' zeros')
    
    # combine the positives with the generated negatives
    x.extend(x_neg[:len(y)])    
    rbps.extend(rbps_neg[:len(y)])  
    y.extend(y_neg[:len(y)])
    print(len(y), ' total; ', 'label balance: ', y.count(1), ' ones, ', y.count(0), ' zeros')
    
    return x, rbps, y

2023-09-25 08:55:45.856854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
home_path = '/home/jovyan'
pos_5p_train = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/upstream-5P-end-transcripts_fasta_train.fa'.format(reads_dataset_name, reads_dataset_preprocessing)
pos_5p_test = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/upstream-5P-end-transcripts_fasta_test.fa'.format(reads_dataset_name, reads_dataset_preprocessing)

pos_3p_train = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/downstream-3P-end-transcripts_fasta_train.fa'.format(reads_dataset_name, reads_dataset_preprocessing)
pos_3p_test = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/downstream-3P-end-transcripts_fasta_test.fa'.format(reads_dataset_name, reads_dataset_preprocessing)

within_train = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/within_transcripts_fasta_train.fa'.format(reads_dataset_name, reads_dataset_preprocessing)
within_test = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/within_transcripts_fasta_test.fa'.format(reads_dataset_name, reads_dataset_preprocessing)

neg_file = home_path + '/decay/decay/data/hg38/transcripts.fa'

rbp_file = home_path + '/decay/decay/rbp_data/AATF.bed'

rep = False # was True --- switch for soft-masked and hard-masked nucleotides 
onehot = False # Return nucleotide characters or one-hot vectors


In [5]:
pos_5p_test

'/home/jovyan/decay/decay/get_fasta/results/hsa_dRNA_HeLa_NoArs_polyA_5P_1/adaptive/adaptive/decay_seqs_fasta/upstream-5P-end-transcripts_fasta_test.fa'

### Load data

In [6]:
import os

if not os.path.exists(dataset_save_name):
    # Load test data
    x_test, rbps_test, y_test = load_data(
        pos_5p_test, pos_3p_test, within_test, 
        rbp_file, neg_file, rep, onehot)

In [7]:
if not os.path.exists(dataset_save_name):
    if onehot:
        n_seqlength, n_features = x_test[0].shape[0], x_test[0].shape[1]
        n_seqlength, n_features
    n_seqlength, n_features = np.shape(x_test)[0], np.shape(x_test)[1]
    n_seqlength, n_features


# Model and training

In [8]:

### Parameters

MODEL_NAME = "simecek/DNADebertaK6c"
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 6
STRIDE = 1

SEED=42
np.random.seed(SEED)

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


## Nucleotide masking

In [10]:
from datasets import Dataset, DatasetDict

if not os.path.exists(dataset_save_name):

    hard_nucleotide_masking = False
    if not hard_nucleotide_masking:
    #     hack to unmask nucleotides
        x_test = np.char.upper(x_test)
    else:
        # no action needed, the tokenizer will map lower case chars to UNK tokens
        pass

    def kmers_strideK(s, k=K):
        return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

    def kmers_stride1(s, k=K):
        return [s[i:i + k] for i in range(0, len(s)-k+1)]

    if (STRIDE == 1):
        kmers = kmers_stride1
    else:
        kmers = kmers_strideK

    # function used for the actual tokenization
    if(K is not None):
        def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])), truncation=True)
    else:
        def tok_func(x): return tokenizer(x["seq"], truncation=True)


    tmp_dict_test = {}
    # TEST
    for index, (x, y) in enumerate(zip(x_test, y_test)):
        tmp_dict_test[f"{index}"] = ("test", y, x)

    df_test = pd.DataFrame.from_dict(tmp_dict_test).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    df_test['seq'] = [''.join(map(str, l)) for l in df_test['seq']]


    ds_test = Dataset.from_pandas(df_test)

    tok_ds_test = ds_test.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds_test = tok_ds_test.rename_columns({'cat':'labels'})


    dds = DatasetDict({
        # 'train': [],
        # 'valid': [],
        'test': tok_ds_test
    })

### Save dataset

In [11]:
from datasets import load_from_disk

if os.path.exists(dataset_save_name):
    print("File exists:", dataset_save_name)
    dds = load_from_disk(dataset_save_name)
else:
    print("File not found:", dataset_save_name)
    dds.save_to_disk(dataset_save_name)
    

File exists: hsa_dRNA_HeLa_NoArs_polyA_5P_1_24n_adaptive/adaptive_test


In [12]:
import evaluate

binary_metrics = evaluate.combine([
    'accuracy',
    'f1',
    'recall',
    'precision',
    #Order of roc_auc matters for logging -> macro first, then weighted
    evaluate.load('roc_auc', average='macro'),
    evaluate.load('roc_auc', average='weighted'),
    evaluate.load("Vlasta/pr_auc"),
])

In [13]:
import pandas as pd
from random import random, randrange
from transformers import AutoModelForSequenceClassification
import torch
from datasets import load_metric


def compute_metrics_binary(eval_preds):
    logits, labels = eval_preds
    prediction_scores = torch.nn.functional.softmax(
        torch.from_numpy(logits).double(), dim=-1).numpy() 
    # predictions = np.argmax(logits, axis=-1) #equivalent
    predictions = np.argmax(prediction_scores, axis=-1)
    return binary_metrics.compute(
        predictions=predictions, 
        references=labels, 
        prediction_scores=prediction_scores[:,1] #taking only prediction percentage for the label 1
    )
    
#TODO dataset multilabel metrics
def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

In [14]:
num_of_labels=2
compute_metrics = compute_metrics_binary if num_of_labels == 2 else compute_metrics_multi

In [15]:
model = torch.load(LOAD_NAME)

In [16]:
from torch.utils.data import DataLoader

# Assuming the 'model' variable contains the loaded PyTorch model
# and the 'datasets' variable is an instance of the Dataset class, tokenized and ready for evaluation

# Load the desired metric, e.g., accuracy
metric = load_metric("accuracy")

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Set the dataset format to PyTorch tensors
eval_dataset = dds["test"] 
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Create a DataLoader for the evaluation dataset
eval_dataloader = DataLoader(eval_dataset, batch_size=16)

predictions_logits = []
# Evaluate the model on the dataset
predictions, labels = [], []
for batch in eval_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    batch_labels = batch['labels'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=batch_labels)
        logits = outputs.logits

    predictions_logits.extend(logits.cpu().numpy())
    batch_predictions = torch.argmax(logits, dim=-1)
    predictions.extend(batch_predictions.cpu().numpy())
    labels.extend(batch_labels.cpu().numpy())

# Compute the evaluation metric
# compute_metrics_binary(eval_preds=(predictions, labels))
eval_metric = metric.compute(predictions=predictions, references=labels)
print(eval_metric)


/tmp/ipykernel_3993/1436649.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


{'accuracy': 0.6759899626660139}


In [17]:
logs = compute_metrics_binary((np.array(predictions_logits), np.array(labels)))
logs

{'accuracy': 0.6759899626660139,
 'f1': 0.6953619518931982,
 'recall': 0.7395801456637493,
 'precision': 0.6561329206711191,
 'rocauc_0_roc_auc': 0.7418106689055893,
 'rocauc_1_roc_auc': 0.7418106689055893,
 'pr_auc': 0.7185300445909809}

In [18]:
from sklearn import metrics

preds = np.argmax(predictions_logits, axis=-1)

In [19]:
confusion_matrix_test = metrics.confusion_matrix(labels, preds)

tn, fp, fn, tp = confusion_matrix_test.ravel()
print(tp + fn, tn + fp)

precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)

print('precision ', precision)
print('recall ', recall)
print('specificity ', specificity)

confusion_matrix_test = confusion_matrix_test.tolist()
confusion_matrix_test.append({'precision': precision, 'recall': recall, 'specificity': specificity})

16339 16339
precision  0.6561329206711191
recall  0.7395801456637493
specificity  0.6123997796682783


count of true negatives is C(0,0), false negatives is C(1,0), true positives is C(1,1) and false positives is C(0,1)

In [20]:
logs = [logs]
logs.append({'confusion_matrix_test' : confusion_matrix_test})

In [21]:
logs

[{'accuracy': 0.6759899626660139,
  'f1': 0.6953619518931982,
  'recall': 0.7395801456637493,
  'precision': 0.6561329206711191,
  'rocauc_0_roc_auc': 0.7418106689055893,
  'rocauc_1_roc_auc': 0.7418106689055893,
  'pr_auc': 0.7185300445909809},
 {'confusion_matrix_test': [[10006, 6333],
   [4255, 12084],
   {'precision': 0.6561329206711191,
    'recall': 0.7395801456637493,
    'specificity': 0.6123997796682783}]}]

In [22]:
import json

with open(LOG_OUTPUT_LOCATION, 'w+') as f:
    f.write(json.dumps(logs))

In [23]:
LOG_OUTPUT_LOCATION

'results_logs/rad5cds_local.max.ds2to11_adaptive_wd0.01_25n.pth_trained_adaptive.adaptive_tested_9.txt'